In [ ]:
import numpy as np
import cv2
import os

In [ ]:
path = 'imgQuery' # Path to the query dataset
orb = cv2.ORB_create(nfeatures=1000) # ORB detector
sift = cv2.SIFT_create(nfeatures=1000) # SIFT detector
img = []
classnames = [] # List of class names

myList = os.listdir(path) # List of all the images in the query dataset

In [ ]:
print(myList)
print("Total Classes Detected:",len(myList)) # Print the total number of classes detected

In [ ]:
for cl in myList: # Loop through all the classes
    imgCur = cv2.imread(f'{path}/{cl}',0) # Read the image
    img.append(imgCur) # Append the image to the list
    classnames.append(os.path.splitext(cl)[0]) # Append the class name to the list

In [ ]:
print(classnames)

In [ ]:
def findDes(images): # Function to find the descriptors of all the images
    desList = []
    for img in images:
        kp, des = orb.detectAndCompute(img,None)    # Find the keypoints and descriptors orb 
        desList.append(des)
    return desList
    

In [ ]:
def findDessift(images): # Function to find the descriptors of all the images
    dessList = []
    for img in images:
        kps, dess = sift.detectAndCompute(img,None) # Find the keypoints and descriptors sift
        dessList.append(dess)
    return dessList

In [ ]:
def findID(img, desList, thres=15): # Function to find the ID of the image
    kp2, des2 = orb.detectAndCompute(img, None)
    bf = cv2.BFMatcher() # Create the BFMatcher object
    matchList = []
    finalVal = -1
    try:
        for des in desList:
            matches = bf.knnMatch(des, des2, k=2)
            good = []
            for m,n in matches:
                if m.distance < 0.75*n.distance:
                    good.append([m])
            matchList.append(len(good))
    except:
        pass
    if len(matchList)!=0:
        if max(matchList) > thres:
            finalVal = matchList.index(max(matchList))
    return finalVal

In [ ]:
def findSID(img,dessList,thres=15): # Function to find the ID of the image
    kps2, dess2 = sift.detectAndCompute(img, None)
    bf = cv2.BFMatcher() # Create the BFMatcher object
    matchLists = []
    finalVal = -1
    try:
        for dess in dessList:
            matches = bf.knnMatch(dess, dess2, k=2)
            good = []
            for m,n in matches:
                if m.distance < 0.75*n.distance:
                    good.append([m])
            matchLists.append(len(good))
    except:
        pass
    if len(matchLists)!=0:
        if max(matchLists) > thres:
            finalVal = matchLists.index(max(matchLists))
    return finalVal

In [ ]:
cap = cv2.VideoCapture(0) # Capture the video from the webcam

In [ ]:
deslist = findDes(img) # Find the descriptors of all the images
desslist = findDessift(img) # Find the descriptors of all the images
print(len(deslist)) # Print the total number of descriptors
print(len(desslist)) # Print the total number of descriptors

while True:
    success,img2 = cap.read() # Read the video
    imgOriginal = img2.copy() # Copy the frame
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY) # Convert the frame to grayscale
 
    id = findID(img2,deslist) # Find the descriptors of the frame
    ids = findSID(img2,desslist) # Find the descriptors of the frame
    if ids != -1:
        cv2.putText(imgOriginal, 'SIFT '+classnames[ids], (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 255), 2)
         
    if id != -1:
         
        cv2.putText(imgOriginal, 'ORB '+classnames[id], (50,100), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0),2)
    
    cv2.imshow('img2',imgOriginal) # Show the original frame
    cv2.waitKey(1) # Wait for 1 millisecond
